Euclid GC Fisher Matrix Code

Author: Alkistis Pourtsidou, ICG Portsmouth

Using part of http://camb.readthedocs.io/en/latest/CAMBdemo.html 

To run this Jupyter notebook you need to have CAMB and the CAMB python package 
installed. In order to install the CAMB python package on your computer follow
the instructions in http://camb.readthedocs.io/en/latest/

In [1]:
%matplotlib inline
import sys, platform, os

from matplotlib import pyplot as plt
import numpy as np

print('Using CAMB installed at '+ os.path.realpath(os.path.join(os.getcwd(),'..')))
sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
import camb
from camb import model, initialpower

Using CAMB installed at /Users/alkistis/workspace


In [2]:
import scipy
from scipy.interpolate import interp1d
from __future__ import division

In [3]:
from scipy import integrate
from scipy import linalg

pi=np.pi

In [4]:
#Fiducial cosmological parameters

hubble=0.67
omegab=0.022445/hubble**2
omegac=0.121203/hubble**2
om0=omegac+omegab
H00=100*hubble
Ass=2.1265e-9
nss=0.96

kpiv=0.05/hubble #in units [h/Mpc]

gamma=0.545

c=3e5

#print om0

In [5]:
#Set up the fiducial cosmology
pars = camb.CAMBparams()
#Set cosmology
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);

In [6]:
#calculate results for these parameters
results = camb.get_results(pars)
#print pars

In [7]:
#Get matter power spectrum at z=0: P(k,z=0)

#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)

#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 200)

#print results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 200)[2]

In [8]:
#Construct P(k,z=0) interpolating function
Pkz0 = interp1d(kh, pk[0])

print Pkz0(0.1)

5714.36444785


In [9]:
#get fiducial σ8 (at z=0)
sig8_fid = results.get_sigma8()[0]
print sig8_fid

0.830287480109


In [10]:
#Redshift bins
zlist = np.arange(0.7,2.1,0.1)
ztest = zlist[0]
Nzbins = len(zlist)

biaslist = [1.083, 1.125, 1.104, 1.126, 1.208, 1.243, 1.282, 1.292, 1.363, 1.497, 1.486, \
            1.491, 1.573, 1.568]

kmin = 0.001
kmax = 0.2

dn3 = [2434.28, 4364.812, 4728.559, 4825.798, 4728.797, 4507.625, 4269.851, 3720.657, 3104.309, \
       2308.975, 1541.831, 1474.707, 893.716, 497.613]

print zlist
print len(zlist), len(dn3)

[ 0.7  0.8  0.9  1.   1.1  1.2  1.3  1.4  1.5  1.6  1.7  1.8  1.9  2. ]
14 14


In [11]:
#Spatially flat Universe

#Define E(z) = H(z)/H0
def Ez(zc):
    return np.sqrt(1-om0+om0*pow(1+zc,3))
def Hz(zc):
    return Ez(zc)*H00

#Define the cosmological distances
def drdz(zp):
    return (c/H00)/Ez(zp)
def rcom(zc):
    return scipy.integrate.romberg(drdz,0,zc)
def DA(zc):
    return rcom(zc)/(1+zc)

print rcom(ztest)
print DA(ztest)

2596.15131772
1527.14783395


In [12]:
#LCDM growth rate
def fg(zz):
    omz=om0*pow(1+zz,3)/pow(Ez(zz),2)
    return pow(omz,gamma)

print fg(ztest)

0.822099034946


In [13]:
#Get the growth factor 
def Dg_dz(zz):
    return -fg(zz)/(1+zz)

def Dgz(zc):
    ans = scipy.integrate.romberg(Dg_dz, 0.0, zc)
    return np.exp(ans)

print Dgz(ztest), Dgz(0)

0.693913449649 1.0


In [14]:
#Construct P_gg(k,μ,z)
def Pgg(kk,mu,zc):
    return pow(bg+fg(zc)*mu**2,2)*pow(Dgz(zc),2)*Pkz0(kk)

In [15]:
#Euclid
Area=15000.0 #deg^2
omegatot = Area*pow(pi/180,2)

Dzbin = 0.1

In [16]:
def photoz(kk,mu,zc):
    sigz = 0.001*(1+zc)
    return np.exp(-pow(kk*mu,2)*pow(c*sigz,2)/pow(Hz(zc),2))

print photoz(0.5,0.5,ztest)

0.200298557962


In [17]:
#survey (bin) volume [Mpc^3]
def dVsurdz(zz):    
    return omegatot*c*pow(rcom(zz),2)/(H00*Ez(zz))
    
def Vsur(zc):
    return scipy.integrate.romberg(dVsurdz,zc-Dzbin/2,zc+Dzbin/2)

def Pshot(zc):
    return Vsur(zc)/Ngal

#effective volume; the hubble**3 factors are for units consistency
def Veff(kk,mu,zc):
    return hubble**3*Vsur(zc)*(Pgg(kk,mu,zc)*photoz(kk,mu,zc)/(Pgg(kk,mu,zc)*photoz(kk,mu,zc)\
                                                     +hubble**3*Pshot(zc)))**2

print ztest, "%.4g" %  Vsur(ztest)

0.7 9.184e+09


In [18]:
#Fisher matrix parameters 
params = ["0:lnH","1:lnDA","2:lnfsig8","3:lnbsig8","4:Ps","5:ns","6:wb","7:wm","8:h"]

In [19]:
#Fisher matrix derivatives 

#z-dependent parameters

def dlnP_dlnfsig8(kk,mu,zc):
    return 2*mu**2*fg(zc)/(bg+mu**2*fg(zc))

def dlnP_dlnbsig8(kk,mu,zc):
    return 2*bg/(bg+mu**2*fg(zc))

def dlnP_dlnDA(kk,mu,zc):
    dk = (kmax-kmin)/400
    return -2.0+4*mu**2*(1-mu**2)*fg(zc)/(bg+mu**2*fg(zc))\
            -kk*(1-mu**2)*(1/Pkz0(kk))*(Pkz0(kk+dk)-Pkz0(kk-dk))/(2*dk)

def dlnP_dlnH(kk,mu,zc):
    dk = (kmax-kmin)/400
    return 1.0+4*mu**2*(1-mu**2)*fg(zc)/(bg+mu**2*fg(zc))\
            +kk*mu**2*(1/Pkz0(kk))*(Pkz0(kk+dk)-Pkz0(kk-dk))/(2*dk)

def dlnP_dPs(kk,mu,zc):
    return 1.0/(Pgg(kk,mu,zc)*photoz(kk,mu,zc))

In [20]:
#shape parameters

# Numerical derivative wrt ns 

step_ns = 0.05*nss
ns_p = nss+step_ns
ns_m = nss-step_ns

#DERIV PLUS
pars = camb.CAMBparams()
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=ns_p, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);
results = camb.get_results(pars)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 200)

sig8_ns_p = results.get_sigma8()[0]
Pkz0_ns_p = interp1d(kh, pk[0])

#DERIV MINUS
pars = camb.CAMBparams()
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=ns_m, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);
results = camb.get_results(pars)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 200)

sig8_ns_m= results.get_sigma8()[0]
Pkz0_ns_m = interp1d(kh, pk[0])

#RESET FIDUCIAL
pars = camb.CAMBparams()
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);
results = camb.get_results(pars)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)

#print sig8_ns_m, sig8_fid, sig8_ns_p
#print results.get_sigma8()[0]

# Numerical derivative wrt Ωbh2 

wb = omegab*pow(hubble,2)
step_wb = 0.05*wb
wb_p = wb+step_wb
wb_m = wb-step_wb

#DERIV PLUS
pars = camb.CAMBparams()
pars.set_cosmology(H0=H00, ombh2=wb_p, omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);
results = camb.get_results(pars)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 200)

sig8_wb_p = results.get_sigma8()[0]
Pkz0_wb_p = interp1d(kh, pk[0])

#DERIV MINUS
pars = camb.CAMBparams()
pars.set_cosmology(H0=H00, ombh2=wb_m, omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);
results = camb.get_results(pars)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 200)

sig8_wb_m= results.get_sigma8()[0]
Pkz0_wb_m = interp1d(kh, pk[0])

#RESET FIDUCIAL
pars = camb.CAMBparams()
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);
results = camb.get_results(pars)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)

#print sig8_wb_m, sig8_fid, sig8_wb_p
#print results.get_sigma8()[0]

# Numerical derivative wrt Ωmh2 

wm = omegab*pow(hubble,2)+omegac*pow(hubble,2)
wc = omegac*pow(hubble,2)
step_wm = 0.05*wc
wm_p = wm+step_wm
wm_m = wb-step_wm

#DERIV PLUS
pars = camb.CAMBparams()
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=wc+step_wm,omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);
results = camb.get_results(pars)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 200)

sig8_wm_p = results.get_sigma8()[0]
Pkz0_wm_p = interp1d(kh, pk[0])

#DERIV MINUS
pars = camb.CAMBparams()
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=wc-step_wm,omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);
results = camb.get_results(pars)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 200)

sig8_wm_m= results.get_sigma8()[0]
Pkz0_wm_m = interp1d(kh, pk[0])

#RESET FIDUCIAL
pars = camb.CAMBparams()
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);
results = camb.get_results(pars)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)

#print sig8_wm_m, sig8_fid, sig8_wm_p
#print results.get_sigma8()[0]

# Numerical derivative wrt h 

step_h = 0.05*hubble
h_p = hubble+step_h
h_m = hubble-step_h

#DERIV PLUS
pars = camb.CAMBparams()
pars.set_cosmology(H0=100*h_p, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);
results = camb.get_results(pars)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 200)

sig8_h_p = results.get_sigma8()[0]
Pkz0_h_p = interp1d(kh, pk[0])

#DERIV MINUS
pars = camb.CAMBparams()
pars.set_cosmology(H0=100*h_m, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);
results = camb.get_results(pars)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 200)

sig8_h_m= results.get_sigma8()[0]
Pkz0_h_m = interp1d(kh, pk[0])

#RESET FIDUCIAL
pars = camb.CAMBparams()
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);
results = camb.get_results(pars)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)

print sig8_h_m, sig8_fid, sig8_h_p
print results.get_sigma8()[0]

0.819922637478 0.830287480109 0.839487889359
0.830287480109


In [21]:
def dlnP_dns(kk):
    return ((Pkz0_ns_p(kk)/pow(sig8_ns_p,2))-(Pkz0_ns_m(kk)/pow(sig8_ns_m,2)))/(2*step_ns)/(Pkz0(kk)/pow(sig8_fid,2))

def dlnP_dwb(kk):
    return ((Pkz0_wb_p(kk)/pow(sig8_wb_p,2))-(Pkz0_wb_m(kk)/pow(sig8_wb_m,2)))/(2*step_wb)/(Pkz0(kk)/pow(sig8_fid,2))

def dlnP_dwm(kk):
    return ((Pkz0_wm_p(kk)/pow(sig8_wm_p,2))-(Pkz0_wm_m(kk)/pow(sig8_wm_m,2)))/(2*step_wm)/(Pkz0(kk)/pow(sig8_fid,2))

def dlnP_dh(kk):
    return ((Pkz0_h_p(kk)/pow(sig8_h_p,2))-(Pkz0_h_m(kk)/pow(sig8_h_m,2)))/(2*step_h)/(Pkz0(kk)/pow(sig8_fid,2))

print dlnP_dns(0.1), dlnP_dwb(0.1), dlnP_dwm(0.1), dlnP_dh(0.1)

-0.459308370485 1.15873078798 -0.698583655411 -0.185234950065


In [22]:
def dF(kk,mu):
    return (1./(8*pi*pi))*pow(kk,2)*deriv_i(kk,mu,zc)*deriv_j(kk,mu,zc)*Veff(kk,mu,zc)    

In [23]:
#2D integration function
def integrate2D(dfun, kgrid, mugrid):
    
    muint = [scipy.integrate.simps(dfun.T[i], mugrid) for i in range(kgrid.size)]
    return scipy.integrate.simps(muint, kgrid)

In [34]:
mugrid = np.linspace(-1., 1., 200) 

In [36]:
fo = open("errors.txt", "w")

#%%time
#   Fisher matrix   # 

Npar = len(params)
#create array of zeros
s = (Npar,Npar)

for zi in range(0,Nzbins):
    
    zc = zlist[zi]
    bg = biaslist[zi]
    zmin = zc-Dzbin/2
    zmax = zc+Dzbin/2
    Ngal = dn3[zi]*Area*Dzbin
    
    kgrid = np.linspace(kmin, kmax, 400)
    K, MU = np.meshgrid(kgrid, mugrid)
    Fishermat = np.zeros(s)
    for i in range(0,Npar):
        def deriv_i(kk,mu,zc):
            if i==0:  return dlnP_dlnH(kk,mu,zc)
            elif i==1:  return dlnP_dlnDA(kk,mu,zc)
            elif i==2:  return dlnP_dlnfsig8(kk,mu,zc)
            elif i==3:  return dlnP_dlnbsig8(kk,mu,zc)
            elif i==4:  return dlnP_dPs(kk,mu,zc)
            elif i==5:  return dlnP_dns(kk)
            elif i==6:  return dlnP_dwb(kk)
            elif i==7:  return dlnP_dwm(kk)
            elif i==8:  return dlnP_dh(kk)
            else: print "Error: index out of range"
        for  j in range(0,Npar):
            if j>=i:
                def deriv_j(kk,mu,zc):
                    if j==0:  return dlnP_dlnH(kk,mu,zc)
                    elif j==1:  return dlnP_dlnDA(kk,mu,zc)
                    elif j==2:  return dlnP_dlnfsig8(kk,mu,zc)
                    elif j==3:  return dlnP_dlnbsig8(kk,mu,zc)
                    elif j==4:  return dlnP_dPs(kk,mu,zc)
                    elif j==5:  return dlnP_dns(kk)
                    elif j==6:  return dlnP_dwb(kk)
                    elif j==7:  return dlnP_dwm(kk)
                    elif j==8:  return dlnP_dh(kk)
                    else: print "Error: index out of range" 
                Fishermat[i][j] = integrate2D(dF(K,MU),kgrid,mugrid)                
            else: Fishermat[i,j] = Fishermat[j,i]
                           
            
    zbin = str(zc)
    
    #marginalised errors
    
    #lnDerr = str("%e" % np.sqrt(linalg.inv(Fishermat)[1,1]))
    #lnHerr = str("%e" % np.sqrt(linalg.inv(Fishermat)[0,0]))
    #lnfsig8err = str("%e" % np.sqrt(linalg.inv(Fishermat)[2,2]))
    #lnbsig8err = str("%e" % np.sqrt(linalg.inv(Fishermat)[3,3]))
    #Pserr = str("%e" % np.sqrt(linalg.inv(Fishermat)[4,4]))  
    #nserr = str("%e" % np.sqrt(linalg.inv(Fishermat)[5,5]))
    #wberr = str("%e" % np.sqrt(linalg.inv(Fishermat)[6,6]))
    #wmerr = str("%e" % np.sqrt(linalg.inv(Fishermat)[7,7]))
    #herr = str("%e" % np.sqrt(linalg.inv(Fishermat)[8,8]))
    
    #un-marginalised errors
        
    lnDerr = str("%e" % (1/np.sqrt(Fishermat[1,1])))
    lnHerr = str("%e" % (1/np.sqrt(Fishermat[0,0])))
    lnfsig8err = str("%e" % (1/np.sqrt(Fishermat[2,2])))
    lnbsig8err = str("%e" % (1/np.sqrt(Fishermat[3,3])))
    Pserr = str("%e" % (1/np.sqrt(Fishermat[4,4])))
    nserr = str("%e" % (1/np.sqrt(Fishermat[5,5])))
    wberr = str("%e" % (1/np.sqrt(Fishermat[6,6])))
    wmerr = str("%e" % (1/np.sqrt(Fishermat[7,7])))
    herr = str("%e" % (1/np.sqrt(Fishermat[8,8])))
    
    print zbin, wmerr, wberr, herr, nserr, lnDerr, lnHerr, lnfsig8err, lnbsig8err, Pserr

    
    outp = zbin + "\t" + wmerr + "\t" + wberr + "\t" + herr + "\t" + nserr + \
    "\t" + lnDerr + "\t" + lnHerr + "\t" + lnfsig8err + "\t" + lnbsig8err + "\t" + Pserr + "\n"
    fo.write(outp)
    
fo.close()    

0.7 1.394355e-03 7.643475e-04 4.770850e-03 7.503232e-03 3.436264e-03 3.344743e-03 6.659988e-03 1.879121e-03 6.385046e+00
0.8 1.247645e-03 6.520364e-04 4.136826e-03 6.716221e-03 2.955718e-03 2.852374e-03 5.779912e-03 1.593738e-03 5.186682e+00
0.9 1.189796e-03 6.318278e-04 3.986959e-03 6.404005e-03 2.871614e-03 2.741492e-03 5.416754e-03 1.556835e-03 4.526602e+00
1.0 1.146388e-03 6.173795e-04 3.877365e-03 6.169703e-03 2.794702e-03 2.682854e-03 5.277436e-03 1.523338e-03 4.257106e+00
1.1 1.108710e-03 6.013655e-04 3.767757e-03 5.966570e-03 2.693204e-03 2.647929e-03 5.336809e-03 1.472183e-03 4.337878e+00
1.2 1.089566e-03 6.015845e-04 3.746624e-03 5.862876e-03 2.675246e-03 2.662064e-03 5.374756e-03 1.471939e-03 4.269770e+00
1.3 1.077241e-03 6.052838e-04 3.747555e-03 5.796057e-03 2.671225e-03 2.694241e-03 5.462405e-03 1.479208e-03 4.253326e+00
1.4 1.087701e-03 6.335393e-04 3.875167e-03 5.851906e-03 2.771159e-03 2.827562e-03 5.662761e-03 1.554620e-03 4.297630e+00
1.5 1.100781e-03 6.623812e-04 4.

In [38]:
print Fishermat[1,1]

22235.2480573


In [39]:
print Fishermat

[[  1.44755188e+04  -1.00691915e+04   3.32735168e+03   2.39930295e+04
    5.52410703e+00  -7.58405828e+03   6.32576017e+03  -3.96931198e+04
    8.11106728e+03]
 [ -1.00691915e+04   2.22352481e+04  -8.35425806e+03  -2.74236403e+04
   -5.84764849e+00   1.01893562e+04  -8.85752488e+03   5.33263553e+04
   -1.08603563e+04]
 [  3.32735168e+03  -8.35425806e+03   3.76352292e+03   1.03862130e+04
    2.41683569e+00  -3.35992115e+03   5.99060270e+03  -1.59754955e+04
    2.75476622e+03]
 [  2.39930295e+04  -2.74236403e+04   1.03862130e+04   5.18431845e+04
    1.25822675e+01  -1.46504117e+04   2.57257477e+04  -7.00060666e+04
    1.21576162e+04]
 [  5.52410703e+00  -5.84764849e+00   2.41683569e+00   1.25822675e+01
    4.29973513e-03  -1.41770875e+00  -3.18504155e+00  -5.50113549e+00
    1.34231998e+00]
 [ -7.58405828e+03   1.01893562e+04  -3.35992115e+03  -1.46504117e+04
   -1.41770875e+00   9.17515507e+03  -2.75893801e+04   4.85908045e+04
   -7.90190162e+03]
 [  6.32576017e+03  -8.85752488e+03   5.